In [1]:
from gensim.models.doc2vec import *

Using TensorFlow backend.


In [3]:
model = Doc2Vec.load('../store/doc2vec_model.mod')

In [8]:
import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer
cohen_vec = cPickle.load(open('../data/vectorizers/cohendata_dedup_5000.p'))

In [10]:
X = cohen_vec
X = map(lambda s : [cohen_vec.idx2word[t] for t in s if t > 0], X)

In [16]:
embedds = []
for sent in X :
    embedds.append(model.infer_vector(sent))

In [17]:
import numpy as np
embedds = np.array(embedds)
embedds.shape

(1767, 600)

In [18]:
import pandas as pd
df = pd.read_csv('../data/files/test_cohen_dedup.csv')

nb_studies = len(df)
H = np.zeros((nb_studies, nb_studies))

cdnos = list(set(df.cdno))
for i in range(nb_studies) :
    H[i, df[df['cdno'] == df['cdno'][i]].index] = 1
    
H[np.arange(nb_studies), np.arange(nb_studies)] = 0

In [19]:
scores = np.dot(embedds, embedds.T)
scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000

In [20]:
K = 10
args = scores.argsort(axis=1)[:, -K:]
S = np.zeros((nb_studies, K))
for i in range(nb_studies) :
    for j in range(K) :
        S[i, j] = H[i, args[i, j]]
np.mean(S.sum(axis=1)/K)

0.48336162988115444

In [21]:
from sklearn.metrics import roc_auc_score
aucs = [0] * nb_studies
for i in range(nb_studies) :
    aucs[i] = roc_auc_score(H[i], scores[i])
print np.mean(aucs)
rocs = {}
for cd in cdnos :
    idxs = df[df.cdno == cd].index
    rocs[cd] = np.mean(np.array(aucs)[idxs])
print rocs

0.726880710625
{'NSAIDS_processed': 0.79684524601110873, 'ACEInhibitors_processed': 0.74680867168079557, 'SkeletalMuscleRelaxants_processed': 0.64665106420559915, 'Triptans_processed': 0.78959677880780887, 'OralHypoglycemics_processed': 0.78008615122866032, 'CalciumChannelBlockers_processed': 0.58316890780535002, 'BetaBlockers_processed': 0.64076287980577185, 'Estrogens_processed': 0.86683070381295213, 'ADHD_processed': 0.68889039803143504, 'Statins_processed': 0.68687647378922556, 'Antihistamines_processed': 0.65508106672380606, 'ProtonPumpInhibitors_processed': 0.75539353919659258, 'Opiods_processed': 0.71804623836626325, 'AtypicalAntipsychotics_processed': 0.7877828436339076}
